# Running a second order Applequist interaction between two water molecules
This tutorial will describe the steps needed to calculate the classical polarizability and hyperpolarizability of two water molecules interacting, using the localized properties of each molecule obtained seperately from quantum mechanics.


------------


# Step 1
## Obtaining the DALTON source code

The DALTON source can be obtained from http://www.daltonprogram.org/. It uses an LGPL license.

-----------

To run a DALTON calculation once it is installed, execute the dalton runscript:

```bash
>>> dalton -get input.dal molecule.mol
```

To see a list of options such as MPI core usage and temporary directory setup, execute:

```bash
>>> dalton
```


#### The following input file should be used for a quadratic response calculation

# dalton.inp

#### The following molecule file specifies one TIP3P water molecule in the xz-plane with its dipole pointing in the positive z-axis direction
## molecule.inp

## Step 2 obtain properties from LoProp

In [5]:
print w.o

0-MOL-OW 0.000000 0.000000 0.000000
